In [2]:
from tensorflow import keras
from keras.layers import Dense, SimpleRNN, Dropout, Input, Flatten, LSTM, Embedding, Bidirectional
import numpy as np
import pandas as pd
import ast
import re

In [3]:
import gensim.downloader as api
model_embedding = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [4]:
model_embedding.most_similar(positive=['имущество'])

[('арестовал', 0.6874980926513672),
 ('делит', 0.6332550048828125),
 ('расходы', 0.6277076005935669),
 ('гранты', 0.6274681091308594),
 ('ребёнка', 0.6251256465911865),
 ('владельца', 0.6227707862854004),
 ('адвоката', 0.6206133961677551),
 ('отбирать', 0.6170918345451355),
 ('арестовали', 0.6161015033721924),
 ('арест', 0.6145174503326416)]

In [5]:
model_embedding.similar_by_vector(model_embedding['гена'])

[('гена', 1.0),
 ('крокодил', 0.6995565295219421),
 ('чебурашка', 0.6883164644241333),
 ('стас', 0.6753256916999817),
 ('виталик', 0.6509150862693787),
 ('михайлов', 0.6437711715698242),
 ('дружок', 0.6290872097015381),
 ('артём', 0.6183868646621704),
 ('вадим', 0.6158268451690674),
 ('никита', 0.6031651496887207)]

In [6]:
model_embedding.most_similar(positive=['гена'])

[('крокодил', 0.6995565891265869),
 ('чебурашка', 0.6883165240287781),
 ('стас', 0.6753256916999817),
 ('виталик', 0.6509150862693787),
 ('михайлов', 0.6437711715698242),
 ('дружок', 0.6290872693061829),
 ('артём', 0.6183868646621704),
 ('вадим', 0.6158269047737122),
 ('никита', 0.6031651496887207),
 ('андрюха', 0.5998855233192444)]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
data = pd.read_csv('/content/drive/MyDrive/datasets/balanced_df.csv', converters={'text':ast.literal_eval})

In [9]:
data

,Review Text,Rating
0,часто хинкали вчера заказать 4 человек хинкали...,1
1,отвратительный работать персонал ресторан понт...,1
2,отвратительный сервис отсутствие гибкость разд...,1
3,добрый день комментарий нет звонить сегоднять...,1
4,ночь 17 06 18 06 заказать салат греческий сала...,1
...,...,...
9505,брать суп брать обязательно заказать пара леп ...,5
9506,замечательный место каждый день ходить обедать...,5
9507,замечательный качество приготовление блюдо пож...,5
9508,понравиться праздничный атмосфера веселие вост...,5


In [10]:
import string
punktuation = string.punctuation
punktuation+='«'+'»'+'—'

In [11]:
def clean_punktuacy(x):
    return [word for word in x if not word in punktuation]

In [12]:
data['Review Text'] =data['Review Text'].astype(str)

In [13]:
data['text'] = data['Review Text'].map(clean_punktuacy)

In [14]:
data

,Review Text,Rating,text
0,часто хинкали вчера заказать 4 человек хинкали...,1,"[ч, а, с, т, о, , х, и, н, к, а, л, и, , в, ..."
1,отвратительный работать персонал ресторан понт...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ..."
2,отвратительный сервис отсутствие гибкость разд...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ..."
3,добрый день комментарий нет звонить сегоднять...,1,"[ , д, о, б, р, ы, й, , д, е, н, ь, , к, о, ..."
4,ночь 17 06 18 06 заказать салат греческий сала...,1,"[н, о, ч, ь, , 1, 7, , 0, 6, , 1, 8, , 0, ..."
...,...,...,...
9505,брать суп брать обязательно заказать пара леп ...,5,"[б, р, а, т, ь, , с, у, п, , б, р, а, т, ь, ..."
9506,замечательный место каждый день ходить обедать...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , м, ..."
9507,замечательный качество приготовление блюдо пож...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , к, ..."
9508,понравиться праздничный атмосфера веселие вост...,5,"[п, о, н, р, а, в, и, т, ь, с, я, , п, р, а, ..."


In [15]:
def insert_digits(x):
    digits = [digit for digit in x if str(digit).isdigit()]

    return digits
def clean_digits(x):
    return [digit for digit in x if not str(digit).isdigit()]

In [16]:
insert_digits([1, 'sdfsdf', 10])

[1, 10]

In [17]:
data['digits'] = data['text'].map(insert_digits)
data['text'] = data['text'].map(clean_digits)

In [18]:
data

,Review Text,Rating,text,digits
0,часто хинкали вчера заказать 4 человек хинкали...,1,"[ч, а, с, т, о, , х, и, н, к, а, л, и, , в, ...",[4]
1,отвратительный работать персонал ресторан понт...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ...",[3]
2,отвратительный сервис отсутствие гибкость разд...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ...",[]
3,добрый день комментарий нет звонить сегоднять...,1,"[ , д, о, б, р, ы, й, , д, е, н, ь, , к, о, ...",[]
4,ночь 17 06 18 06 заказать салат греческий сала...,1,"[н, о, ч, ь, , , , , , з, а, к, а, з, а, ...","[1, 7, 0, 6, 1, 8, 0, 6]"
...,...,...,...,...
9505,брать суп брать обязательно заказать пара леп ...,5,"[б, р, а, т, ь, , с, у, п, , б, р, а, т, ь, ...",[]
9506,замечательный место каждый день ходить обедать...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , м, ...",[]
9507,замечательный качество приготовление блюдо пож...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , к, ...",[1]
9508,понравиться праздничный атмосфера веселие вост...,5,"[п, о, н, р, а, в, и, т, ь, с, я, , п, р, а, ...",[]


In [19]:
mx = 0
mn = 1001010010
len_str = []
for i in range(1713):
    if len(data['text'][i]) < mn:
        mn = len(data['text'][i])
    if len(data['text'][i]) > mx:
        mx = len(data['text'][i])
    if len(data['text'][i]) == 1:
        print(data['text'][i])
        print(i)
    len_str.append(len(data['text'][i]))
mn, mx

[' ']
1566


(0, 6497)

In [20]:
def normalize_len_text(x):
    if len(x) < 654:
        return x + ['стоп_слово'] * (654-len(x))
    return x

In [21]:
data['text'] = data['text'].map(normalize_len_text)

In [22]:
data

,Review Text,Rating,text,digits
0,часто хинкали вчера заказать 4 человек хинкали...,1,"[ч, а, с, т, о, , х, и, н, к, а, л, и, , в, ...",[4]
1,отвратительный работать персонал ресторан понт...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ...",[3]
2,отвратительный сервис отсутствие гибкость разд...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ...",[]
3,добрый день комментарий нет звонить сегоднять...,1,"[ , д, о, б, р, ы, й, , д, е, н, ь, , к, о, ...",[]
4,ночь 17 06 18 06 заказать салат греческий сала...,1,"[н, о, ч, ь, , , , , , з, а, к, а, з, а, ...","[1, 7, 0, 6, 1, 8, 0, 6]"
...,...,...,...,...
9505,брать суп брать обязательно заказать пара леп ...,5,"[б, р, а, т, ь, , с, у, п, , б, р, а, т, ь, ...",[]
9506,замечательный место каждый день ходить обедать...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , м, ...",[]
9507,замечательный качество приготовление блюдо пож...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , к, ...",[1]
9508,понравиться праздничный атмосфера веселие вост...,5,"[п, о, н, р, а, в, и, т, ь, с, я, , п, р, а, ...",[]


In [23]:
def sentence_to_vector(x):
    sequence_embeddings = np.zeros(100*110)
    count_words = 0
    count_words_to_sentense = 0
    for i in range(654):
        try:
            sequence_embeddings[count_words_to_sentense: count_words_to_sentense + 100] += model_embedding[x[i]]
        except:
            pass


        count_words += 1
        if count_words == 6:
            count_words = 0
            count_words_to_sentense += 100
    return sequence_embeddings / 10

In [24]:
sentense = 'я смотрю фильм'.split()
s = np.zeros(200)
for i in range(3):
    s[100:] += model_embedding[sentense[i]]
s

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [25]:
data['vectors'] = data['text'].map(sentence_to_vector)

In [26]:
data

,Review Text,Rating,text,digits,vectors
0,часто хинкали вчера заказать 4 человек хинкали...,1,"[ч, а, с, т, о, , х, и, н, к, а, л, и, , в, ...",[4],"[0.2029793018475175, 0.042180997133255, -0.190..."
1,отвратительный работать персонал ресторан понт...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ...",[3],"[0.21914580054581165, -0.008006203174591064, -..."
2,отвратительный сервис отсутствие гибкость разд...,1,"[о, т, в, р, а, т, и, т, е, л, ь, н, ы, й, , ...",[],"[0.21914580054581165, -0.008006203174591064, -..."
3,добрый день комментарий нет звонить сегоднять...,1,"[ , д, о, б, р, ы, й, , д, е, н, ь, , к, о, ...",[],"[0.21175400167703629, 0.00773649699985981, -0...."
4,ночь 17 06 18 06 заказать салат греческий сала...,1,"[н, о, ч, ь, , , , , , з, а, к, а, з, а, ...","[1, 7, 0, 6, 1, 8, 0, 6]","[0.18074140176177025, 0.050900998711586, -0.05..."
...,...,...,...,...,...
9505,брать суп брать обязательно заказать пара леп ...,5,"[б, р, а, т, ь, , с, у, п, , б, р, а, т, ь, ...",[],"[0.007444401271641254, 0.049562299624085426, -..."
9506,замечательный место каждый день ходить обедать...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , м, ...",[],"[0.03752590417861938, 0.11854559779167176, -0...."
9507,замечательный качество приготовление блюдо пож...,5,"[з, а, м, е, ч, а, т, е, л, ь, н, ы, й, , к, ...",[1],"[0.03752590417861938, 0.11854559779167176, -0...."
9508,понравиться праздничный атмосфера веселие вост...,5,"[п, о, н, р, а, в, и, т, ь, с, я, , п, р, а, ...",[],"[0.2328125, -0.014152202010154723, -0.23314549..."


In [27]:
len(data['vectors'][0])

11000

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['vectors'], data['Rating'], test_size=0.2, random_state=42)

In [30]:
def balancer(x, y):
    dataset = pd.DataFrame(x)
    dataset['target'] = y
    dataset = dataset.append(dataset[dataset['target'] == 1])
    dataset = dataset.append(dataset[dataset['target'] == 1])
    dataset = dataset.append(dataset[dataset['target'] == 0])
    dataset = dataset.append(dataset[dataset['target'] == 0])
    return dataset

df = balancer(X_train, y_train)
df

<ipython-input-30-44ad58d4eca6>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 1])
<ipython-input-30-44ad58d4eca6>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 1])
<ipython-input-30-44ad58d4eca6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 0])
<ipython-input-30-44ad58d4eca6>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(dataset[dataset['target'] == 0])


,vectors,target
1937,"[0.2801969051361084, 0.09749380201101303, -0.1...",2
4745,"[-0.03689660485833883, -0.03808180242776871, -...",3
4965,"[0.005498201027512551, 0.1590184010565281, -0....",3
2760,"[0.06432440225034952, 0.2102908030152321, -0.2...",2
3509,"[0.22313800156116487, 0.01602119691669941, -0....",2
...,...,...
1184,"[-0.024973001331090927, 0.05504249930381775, -...",1
769,"[0.393719401396811, -0.0773528054356575, -0.24...",1
1685,"[0.04225149750709534, -7.380247116088867e-05, ...",1
466,"[0.003692000359296799, 0.09133419841527939, -0...",1


In [31]:
X_train = np.stack(df['vectors'].values).round(2)
y_train = df.target

In [32]:
len(data['vectors'][0])

11000

In [43]:
def change_y(x):
    y_return = [0]*6
    y_return[x] = 1
    return np.array(y_return)

In [44]:
y_train = df['target'].map(change_y)
y_train = np.stack(y_train.values)
y_train.shape

(12135, 6)

In [45]:
X_train

array([[ 0.28,  0.1 , -0.18, ...,  0.  ,  0.  ,  0.  ],
       [-0.04, -0.04, -0.22, ...,  0.  ,  0.  ,  0.  ],
       [ 0.01,  0.16, -0.25, ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.04, -0.  , -0.27, ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.09, -0.32, ...,  0.  ,  0.  ,  0.  ],
       [-0.04, -0.04, -0.22, ...,  0.  ,  0.  ,  0.  ]])

In [48]:
X_test = np.stack(X_test).round(2)
y_test = df['target'].map(change_y)
y_test = np.stack(y_test)
y_test

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [54]:
model = keras.Sequential()

model.add(Dense(300, activation='tanh'))

model.add(Dense(200, activation='tanh'))

model.add(Dense(100, activation='tanh'))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=[keras.metrics.Accuracy()])

In [55]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
380/380 [==============================] - 7s 5ms/step - loss: 1.3555 - accuracy: 0.0000e+00
Epoch 2/100
380/380 [==============================] - 2s 4ms/step - loss: 1.3104 - accuracy: 0.0000e+00
Epoch 3/100
380/380 [==============================] - 2s 4ms/step - loss: 1.2689 - accuracy: 0.0000e+00
Epoch 4/100
380/380 [==============================] - 2s 4ms/step - loss: 1.2414 - accuracy: 0.0000e+00
Epoch 5/100
380/380 [==============================] - 2s 5ms/step - loss: 1.1901 - accuracy: 0.0000e+00
Epoch 6/100
380/380 [==============================] - 2s 5ms/step - loss: 1.1562 - accuracy: 0.0000e+00
Epoch 7/100
380/380 [==============================] - 2s 4ms/step - loss: 1.1022 - accuracy: 0.0000e+00
Epoch 8/100
380/380 [==============================] - 2s 4ms/step - loss: 1.0596 - accuracy: 0.0000e+00
Epoch 9/100
380/380 [==============================] - 2s 4ms/step - loss: 1.0004 - accuracy: 0.0000e+00
Epoch 10/100
380/380 [==============================] -

In [56]:
def max_index(x):
    for i in range(len(x)):
        if x[i] == max(x):
            return i

In [57]:
def check_accuracy(y_pred, y_test):
    count_all = 0
    count_true = 0
    for i in range(len(y_pred)):
        if max_index(y_pred[i]) == max_index(y_test[i]):
            count_true += 1
        count_all += 1
    return count_true / count_all

In [58]:
y_pred = model.predict(X_test)

60/60 [==============================] - 0s 3ms/step


In [59]:
check_accuracy(y_pred, y_test)

0.20609884332281808

In [ ]:
X_test_work = np.array([[]])
X_test_work_common = np.array([[[]]])
for item in range(len(X_test)):
    for i in range(0, 500, 100):
        X_test_work = np.append(X_test_work, X_test[item][i:-(500-i)])
    X_test_work_common = np.append(X_test_work_common, X_test_work)
    X_test_work = np.array([[]])
X_test_work_common = X_test_work_common.reshape(len(X_test), 5, 10500)

In [ ]:
X_train_work = np.array([[]])
X_train_work_common = np.array([[[]]])
for item in range(len(X_train)):
    for i in range(0, 500, 100):
        X_train_work = np.append(X_train_work, X_train[item][i:-(500-i)])
    X_train_work_common = np.append(X_train_work_common, X_train_work)
    X_train_work = np.array([[]])
X_train_work_common = X_train_work_common.reshape(len(X_train), 5, 10500)

In [ ]:
model_rnn = keras.Sequential()



model_rnn.add(LSTM(1024, activation='tanh', return_sequences=True))
model_rnn.add(LSTM(1024, activation='tanh', return_sequences=False))
model_rnn.add(Dense(1024, activation='tanh'))
model_rnn.add(Dense(512, activation='tanh'))
model_rnn.add(Dense(256, activation='tanh'))
model_rnn.add(Dense(3, activation='softmax'))

model_rnn.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001))

In [ ]:
model_rnn.fit(X_train_work_common, y_train, epochs=15)

In [ ]:
y_pred_rnn = model_rnn.predict(X_test_work_common)

In [ ]:
y_pred_rnn

In [ ]:
check_accuracy(y_pred_rnn, y_test)

In [ ]:
model_rnn.save('Model_rnn_10epochs_512x512x3')